# Merging and exploration of individual stock beta values from Wharton databases.
1) Import large csv files then merge on premno (A type of stock ID).
2) Pivot table based on date.
3) Convert date column to 'datetime'
4) Create histogram with quarter (3 month) sized bins. How many companies have beta values here?
5) How many companies are there in each bin with beta and/or price > 0? Can I remove companies with negative betas?
6) What is the largest market cap for each date?
7) Based on this I may be able to safely calculate historical weighted average beta (weighted by market cap)


In [1]:
# TODO: I need to fix the datetime conversion, then I can experiment with plotting.

# Import python classes
import numpy as np
import pandas as pd


In [2]:
mcap_raw = pd.read_csv(r"C:\Users\Edmund\OMSA\MGT6203\group project\datasets\market cap\market_cap.csv")
display(mcap_raw.head())
betas_raw = pd.read_csv(r"C:\Users\Edmund\OMSA\MGT6203\group project\datasets\market_betas_all_data\raw_market_betas.csv")
display(betas_raw.head())
# permno_ticker_translations = pd.read_csv(r"C:\Users\Edmund\OMSA\MGT6203\group project\datasets\Permno_ticker_translations\permno_ticker_translations.csv")
# display(permno_ticker_translations.head())

# what are the shapes of these dataframes?
print(mcap_raw.shape)
print(betas_raw.shape)

,PERMNO,date,TICKER,COMNAM,PRC,SHROUT
0,10000,12/31/1985,NaN,NaN,NaN,NaN
1,10000,01/31/1986,OMFGA,OPTIMUM MANUFACTURING INC,-4.3750,3680.0
2,10000,02/28/1986,OMFGA,OPTIMUM MANUFACTURING INC,-3.2500,3680.0
3,10000,03/31/1986,OMFGA,OPTIMUM MANUFACTURING INC,-4.4375,3680.0
4,10000,04/30/1986,OMFGA,OPTIMUM MANUFACTURING INC,-4.0000,3793.0


,permno,yyyymmdd,bswa32,sd0111
0,10000,1986-01-31,1.04,NaN
1,10000,1986-02-28,0.53,NaN
2,10000,1986-03-31,0.55,NaN
3,10000,1986-04-30,0.33,NaN
4,10000,1986-05-30,0.26,NaN


(4169888, 6)
(3676436, 4)


In [3]:
# The mcap_raw has more rows so let's use it as the baseline.
# I need to put dates in the same format before merging on date and permno
print(type(mcap_raw['date'].loc[0]))
print(mcap_raw['date'].loc[0])
print(type(betas_raw['yyyymmdd'].loc[0]))
print(betas_raw['yyyymmdd'].loc[0])

def mcap_date_convert(mcap_raw): # assumes input is mm-dd-yyyy , convert to yyyy-mm-dd.
    dates_converted = []
    for date in mcap_raw['date']:
        #print(date.split('-'))
        mm, dd, yyyy = date.split('/') # split on dash.
        dates_converted.append(yyyy+'-'+mm+'-'+dd)
    return dates_converted

mcap_raw['date'] = mcap_date_convert(mcap_raw)
display(mcap_raw.head())
# mcaps_and_betas_raw = mcap_raw.merge(betas_raw, left_on='PERMNO', right_on = "permno", how='left') # mer
# display(mcaps_and_betas_raw.head())

<class 'str'>
12/31/1985
<class 'str'>
1986-01-31


,PERMNO,date,TICKER,COMNAM,PRC,SHROUT
0,10000,1985-12-31,NaN,NaN,NaN,NaN
1,10000,1986-01-31,OMFGA,OPTIMUM MANUFACTURING INC,-4.3750,3680.0
2,10000,1986-02-28,OMFGA,OPTIMUM MANUFACTURING INC,-3.2500,3680.0
3,10000,1986-03-31,OMFGA,OPTIMUM MANUFACTURING INC,-4.4375,3680.0
4,10000,1986-04-30,OMFGA,OPTIMUM MANUFACTURING INC,-4.0000,3793.0


In [5]:
mcap_betas_raw = pd.merge(mcap_raw, betas_raw, how='left', left_on=['PERMNO','date'], right_on = ['permno','yyyymmdd']) #leftmerge where perno and date are equal
display(mcap_betas_raw.head())

,PERMNO,date,TICKER,COMNAM,PRC,SHROUT,permno,yyyymmdd,bswa32,sd0111
0,10000,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,1986-01-31,OMFGA,OPTIMUM MANUFACTURING INC,-4.3750,3680.0,10000.0,1986-01-31,1.04,NaN
2,10000,1986-02-28,OMFGA,OPTIMUM MANUFACTURING INC,-3.2500,3680.0,10000.0,1986-02-28,0.53,NaN
3,10000,1986-03-31,OMFGA,OPTIMUM MANUFACTURING INC,-4.4375,3680.0,10000.0,1986-03-31,0.55,NaN
4,10000,1986-04-30,OMFGA,OPTIMUM MANUFACTURING INC,-4.0000,3793.0,10000.0,1986-04-30,0.33,NaN


In [7]:
mcap_betas = mcap_betas_raw[['PERMNO', 'date', 'TICKER', 'COMNAM', 'PRC', 'SHROUT','bswa32']] # only keep columns of interest
# convert string date to datetime for plotting
# mcap_betas['date'] = 
mcap_betas['date'] = pd.to_datetime(mcap_betas['date'], format='%Y-%m-%d' , errors='ignore') # convert 'date' to datetime so plotting can be done.
# print(type(mcap_betas['date'].loc[0]))
print(mcap_betas['date'].dtype)
display(mcap_betas.head())

datetime64[ns]


C:\Users\Edmund\AppData\Local\Temp\ipykernel_24012\3247155042.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcap_betas['date'] = pd.to_datetime(mcap_betas['date'], format='%Y-%m-%d' , errors='ignore') # convert 'date' to datetime so plotting can be done.


,PERMNO,date,TICKER,COMNAM,PRC,SHROUT,bswa32
0,10000,1985-12-31,NaN,NaN,NaN,NaN,NaN
1,10000,1986-01-31,OMFGA,OPTIMUM MANUFACTURING INC,-4.3750,3680.0,1.04
2,10000,1986-02-28,OMFGA,OPTIMUM MANUFACTURING INC,-3.2500,3680.0,0.53
3,10000,1986-03-31,OMFGA,OPTIMUM MANUFACTURING INC,-4.4375,3680.0,0.55
4,10000,1986-04-30,OMFGA,OPTIMUM MANUFACTURING INC,-4.0000,3793.0,0.33


In [17]:
# make histograms to get a sense of the price, shares outstanding, and betas:
# mcap_betas.plot(x = 'date', y = 'PRC') # Why are prices negative?
# mcap_betas.plot(x = 'date', y = 'SHROUT')

print(len(mcap_betas)) # how many rows in the whole dataframe? 
# What if we remove rows where PRC and SHROUT are NaN?
mb_no_nan = mcap_betas.dropna(subset=['PRC','SHROUT'])
print(len(mb_no_nan))
# What is the minimum 'SHROUT' value?
print(min(mb_no_nan['SHROUT']))
# Negative PRC values actually represent negative bid/ask averages. For this analysis I will just remove the negative signs
mb_no_nan['PRC'] = mb_no_nan['PRC'].abs()

# now include column for market cap (price * shares outstanding)
mb_no_nan['MCAP'] = mb_no_nan['PRC'] * mb_no_nan['SHROUT']
display(mb_no_nan.head())

4169888
4039170
0.0


C:\Users\Edmund\AppData\Local\Temp\ipykernel_24012\2939315058.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb_no_nan['PRC'] = mb_no_nan['PRC'].abs()
C:\Users\Edmund\AppData\Local\Temp\ipykernel_24012\2939315058.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb_no_nan['MCAP'] = mb_no_nan['PRC'] * mb_no_nan['SHROUT']


,PERMNO,date,TICKER,COMNAM,PRC,SHROUT,bswa32,MCAP
1,10000,1986-01-31,OMFGA,OPTIMUM MANUFACTURING INC,4.37500,3680.0,1.04,16100.00000
2,10000,1986-02-28,OMFGA,OPTIMUM MANUFACTURING INC,3.25000,3680.0,0.53,11960.00000
3,10000,1986-03-31,OMFGA,OPTIMUM MANUFACTURING INC,4.43750,3680.0,0.55,16330.00000
4,10000,1986-04-30,OMFGA,OPTIMUM MANUFACTURING INC,4.00000,3793.0,0.33,15172.00000
5,10000,1986-05-30,OMFGA,OPTIMUM MANUFACTURING INC,3.10938,3793.0,0.26,11793.87834


In [18]:
print(mb_no_nan.shape)

(4039170, 8)
